In [1]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"
from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("multi-qa-distilbert-cos-v1")

In [2]:
user_question = "I just discovered the course. Can I still join it?"

query_embed_1 = model.encode(user_question)

In [3]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [4]:
filtered_documents = []

for doc in documents:
    if doc["course"] == "machine-learning-zoomcamp":
        filtered_documents.append(doc)

In [5]:
embed_list = []

for doc in filtered_documents:
    question = doc["question"]
    text = doc["text"]
    qa_text = f"{question}: {text}"
    embed_list.append(model.encode(qa_text))

In [6]:
import numpy as np

X = np.array(embed_list)
X.shape

(375, 768)

In [7]:
X[0,:].dot(X[0,:])

0.9999999

In [8]:
scores = X.dot(query_embed_1)

In [9]:
max(scores)

0.65120065

In [10]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
results = search_engine.search(query_embed_1, num_results=5)

In [11]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [12]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [13]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [14]:
from tqdm.auto import tqdm

In [15]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [16]:
def question_vector(q):
    question = q['question']

    query_embed = model.encode(question)

    return search_engine.search(query_embed, num_results=5)

In [17]:
evaluate(ground_truth, question_vector)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9415300546448088, 'mrr': 0.8523679417122041}

In [28]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            # "question_vector": {
            #     "type": "dense_vector",
            #     "dims": 768,
            #     "index": True,
            #     "similarity": "cosine"
            # },
            # "text_vector": {
            #     "type": "dense_vector",
            #     "dims": 768,
            #     "index": True,
            #     "similarity": "cosine"
            # },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [27]:
for doc in tqdm(filtered_documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_text_vector'] = model.encode(qt)

  0%|          | 0/375 [00:00<?, ?it/s]

In [29]:
for doc in tqdm(filtered_documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [21]:
knn = {
        "field": 'question_text_vector',
        "query_vector": query_embed_1,
        "k": 5,
        "num_candidates": 10000,
        # "filter": {
        #     "term": {
        #         "course": course
        #     }
        # }
    }

search_query = {
    "knn": knn,
    "_source": ["text", "section", "question", "course", "id"]
}

es_results = es_client.search(
    index=index_name,
    body=search_query
)

result_docs = []

for hit in es_results['hits']['hits']:
    result_docs.append(hit['_source'])

In [30]:
def elastic_search_knn(vector):
    knn = {
        "field": "question_text_vector",
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        # "filter": {
        #     "term": {
        #         "course": course
        #     }
        # }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [31]:
def question_vector_knn(q):
    question = q['question']

    v_q = model.encode(question)

    return elastic_search_knn(v_q)

In [32]:
evaluate(ground_truth, question_vector_knn)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9404371584699454, 'mrr': 0.8517577413479053}